In [ ]:
# ! pip install transformers datasets

In [ ]:
from datasets import load_dataset

# Load the CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

In [ ]:
from transformers import T5Tokenizer

# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Tokenize the data
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=150, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the preprocessing function to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [ ]:
from transformers import T5Tokenizer

# Load T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Tokenize the data
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=150, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the preprocessing function to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [ ]:
from pdfminer.high_level import extract_text_to_fp
from io import StringIO

def extract_text_from_pdf(pdf_path, start_page, end_page):
    text = ""
    output = StringIO()
    with open(pdf_path, 'rb') as f:
        extract_text_to_fp(f, output, page_numbers=range(start_page-1, end_page))
        text = output.getvalue()
    return text

# Define the path to the PDF and the pages to extract
pdf_path = "looma_sample_book.pdf"  # Update this to the path of your PDF file
start_page = 1  # Update this to the start page of Chapter 1
end_page = 2  # Update this to the end page of Chapter 1

# Extract text from the specified pages
chapter_text = extract_text_from_pdf(pdf_path, start_page, end_page)
print("Extracted Chapter Text:")
print(chapter_text)


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model and tokenizer
model_path = "./results/t5_fine_tuned"  # Path to the fine-tuned model
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Function to generate a summary
def generate_summary(chapter_text):
    inputs = tokenizer("summarize: " + chapter_text, return_tensors='pt', max_length=512, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Generate and print the summary for the extracted chapter text
summary = generate_summary(chapter_text)
print("Summary:")
print(summary)


In [4]:
import fitz  # PyMuPDF
import pytesseract

# Load PDF
pdf_path = "MATH_76_Paper.pdf"  # Replace with the path to your PDF
pdf_document = fitz.open(pdf_path)

def extract_text_from_pdf(pdf_document, start_page, end_page):
    text_by_page = []
    for page_num in range(start_page - 1, end_page):
        page = pdf_document.load_page(page_num)
        text = page.get_text()
        if not text.strip():  # If no text found, use OCR
            images = convert_from_path(pdf_path, first_page=page_num+1, last_page=page_num+1)
            text = pytesseract.image_to_string(images[0])
        text_by_page.append(text)
    return text_by_page

# Extract text for Chapter 1 (from page 1 to 14)
chapter_1_text = extract_text_from_pdf(pdf_document, 0, 1)
chapter_1_text = "\n".join(chapter_1_text)


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "meta-llama3"  # Replace with the actual model name if necessary
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


OSError: meta-llama3 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
def generate_summary(text, max_length=150, num_beams=5):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    summary_ids = model.generate(inputs.input_ids, max_length=max_length, num_beams=num_beams, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Generate summary for Chapter 1
chapter_1_summary = generate_summary(chapter_1_text)

print(f"Summary of Chapter 1:\n{chapter_1_summary}")
